# In this notebook I will create 3d interpolations of the mach probe measurements.

In [10]:
import numpy as np
from scipy.interpolate import griddata

import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

In [11]:
from pyvisfile.vtk import write_structured_grid
from pytools.obj_array import make_obj_array

In [12]:
import sys
sys.path.append('../../read_from_sql/')
import read_from_sql

In [39]:
sys.path.append('/Users/vonderlinden2/rsx_analysis/mach_probe_analysis')
import ion_current_to_mach_number as ic_to_mach
sys.path.append('/Users/vonderlinden2/rsx_analysis/time_alignment/source')
import absolute_times as at

In [13]:
database = '/Users/vonderlinden2/rsx_analysis/shots_database/source/shots.db'
table = 'Shots'

In [25]:
z_direction_1, z_direction_2 = 0, 180
y_direction_1, y_direction_2 = 90, 270

In [42]:
angle_signs = {0: 1,
               180: -1,
               90: -1,
               0: 1}

# Set minimum spectral density

In [14]:
min_spectral_density = 1.6e-8

# Collect suitable shots from the plane_campaign_1, plane_campaign_2, and line_campaign 

## plane_campaign_1

In [28]:
condition_z_0419 = ("campaigns = 'mach_probe_plane_campaign_1'"
                    " AND fiducial_pre_crowbar_gyration_spectral_density > "
                    + str(min_spectral_density) +
                    " AND mach_signals_exist = 1"
                    " AND (mach_orientation = " + str(z_direction_1) +
                    " OR mach_orientation = " + str(z_direction_2) + ")")

condition_y_0419 = ("campaigns = 'mach_probe_plane_campaign_1'"
                    " AND fiducial_pre_crowbar_gyration_spectral_density > "
                    + str(min_spectral_density) +
                    " AND mach_signals_exist = 1"
                    " AND (mach_orientation = " + str(y_direction_1) +
                    " OR mach_orientation = " + str(y_direction_2) + ")")

In [29]:
cursor, connection = read_from_sql.cursor_with_rows(condition_z_0419,
                                                    database,
                                                    table)
z_0419_shots = cursor.fetchall()
cursor.close()
connection.close()

In [30]:
cursor, connection = read_from_sql.cursor_with_rows(condition_y_0419,
                                                    database,
                                                    table)
y_0419_shots = cursor.fetchall()
cursor.close()
connection.close()

## plane_campaign_2

In [35]:
condition_z_302 = ("campaigns = 'mach_probe_plane_campaign_2'"
                   " AND fiducial_pre_crowbar_gyration_spectral_density > "
                   + str(min_spectral_density) +
                   " AND mach_signals_exist = 1"
                   " AND (mach_orientation = " + str(z_direction_1) +
                   " OR mach_orientation = " + str(z_direction_2) + ")")

cursor, connection = read_from_sql.cursor_with_rows(condition_z_302,
                                                    database,
                                                    table)
z_0302_shots = cursor.fetchall()
cursor.close()
connection.close()

## line_campaign_1

In [38]:
condition_z_0249 = ("campaigns = 'mach_probe_line_campaign_1'"
                    " AND fiducial_pre_crowbar_gyration_spectral_density > "
                    + str(min_spectral_density) +
                    " AND mach_signals_exist = 1"
                    " AND (mach_orientation = " + str(z_direction_1) +
                    " OR mach_orientation = " + str(z_direction_2) + ")")

cursor, connection = read_from_sql.cursor_with_rows(condition_z_0249,
                                                    database,
                                                    table)

z_0249_shots = cursor.fetchall()
cursor.close()
connection.close()

# Define functions

In [40]:
def times_to_indexes(time, times):
    r"""
    """
    indexes = np.searchsorted(time, times)
    for i, index in enumerate(indexes):
        indexes[i] = (index-1 +
                      np.argmin([np.abs(times[i] - time[index - 1]),
                                 np.abs(times[i] - time[index]),
                                 np.abs(times[i] - time[index + 1])]))
    return indexes

In [86]:
def run_mach_analysis(shots, time_points, angle_signs):
    mach_out, x_out, y_out, z_out = [], [], [], []
    for shot in shots:

        shot_number = shot['shot']
        times = at.absolute_times(shot_number, shot, [],
                                  number_of_delays=time_points)
        (mach, time,
         r_background_std,
         l_background_std) = ic_to_mach.mach_number(shot_number, 
                                                    mach_r_node_name=shot['mach_r_node'],
                                                    mach_l_node_name=shot['mach_l_node'])
        mach = mach*angle_signs[shot['mach_orientation']]
        indexes = times_to_indexes(time, times)
        mach_out.append(mach[indexes])
        x_out.append(shot['mach_x'])
        y_out.append(shot['mach_y'])
        z_out.append(shot['mach_z'])
    x_out = np.asarray(x_out)
    y_out = np.asarray(y_out)
    z_out = np.asarray(z_out)
    mach_out = np.asarray(mach_out)
    mach_out = np.swapaxes(mach_out, 0, 1)
    measurements = {'a_out': mach_out,
                    'x_out': x_out,
                    'y_out': y_out,
                    'z_out': z_out}
    return measurements

In [87]:
mach_y_0419_measurements = run_mach_analysis(z_0419_shots,
                                             21, 
                                             angle_signs) 
mach_z_0419_measurements = run_mach_analysis(y_0419_shots,
                                             21, 
                                             angle_signs)
mach_y_0302_measurements = run_mach_analysis(z_0302_shots,
                                             21, 
                                             angle_signs)
mach_y_0219_measurements = run_mach_analysis(z_0249_shots,
                                             21, 
                                             angle_signs)

In [88]:
mach_y_0419_measurements['a_out'].shape

(21, 246)